In [79]:
import warnings
import itertools
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression, LinearRegression, ElasticNet, SGDRegressor

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from xgboost import XGBClassifier

from sklearn.metrics import confusion_matrix

In [37]:
df_train = pd.read_csv('../data/raw/Data_Set_Loan_predit/train_data_set.csv')
df_test = pd.read_csv('../data/raw/Data_Set_Loan_predit/test_data_set.csv')
# pd.read_csv('../data/raw/Data_Set_Loan_predit/_data_set.csv')
df_test["Loan_Status"] = "X"
data_preprocess = pd.concat([df_train, df_test]).reset_index()
data_preprocess.drop('index', axis=1, inplace=True)
# data_preprocess['Dependents'].fillna(0, inplace=True)

In [38]:
data_preprocess.dropna(inplace=True)

data_preprocess.reset_index(inplace=True)
data_preprocess.drop('index', axis=1, inplace=True)

data_preprocess["Married"] = data_preprocess["Married"].apply(lambda x: 1 if x == 'Yes' else 0)
data_preprocess["Education"] = data_preprocess["Education"].apply(lambda x: 1 if x == 'Graduate' else 0)
data_preprocess["Self_Employed"]  = data_preprocess["Self_Employed"].apply(lambda x: 1 if x == 'No' else 0)
data_preprocess["Gender"] = data_preprocess["Gender"].apply(lambda x: 1 if x == 'Male' else 0)
data_preprocess["Dependents"] = data_preprocess["Dependents"].apply(lambda x: 3 if x == '3+' else int(x))


In [71]:
model_data = pd.get_dummies(data_preprocess.loc[:,"Gender":"Property_Area"])

In [43]:
model_data
test_data = model_data.iloc[loan_data2['Loan_Amount_Term'].isnull()]
train_data = loan_data2[loan_data2['Loan_Amount_Term'].notnull()]

In [44]:
data_preprocess.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 769 entries, 0 to 768
Data columns (total 13 columns):
Loan_ID              769 non-null object
Gender               769 non-null int64
Married              769 non-null int64
Dependents           769 non-null int64
Education            769 non-null int64
Self_Employed        769 non-null int64
ApplicantIncome      769 non-null int64
CoapplicantIncome    769 non-null float64
LoanAmount           769 non-null float64
Loan_Amount_Term     769 non-null float64
Credit_History       769 non-null float64
Property_Area        769 non-null object
Loan_Status          769 non-null object
dtypes: float64(4), int64(6), object(3)
memory usage: 78.2+ KB


In [76]:
# help(data_preprocess.reset_index)

In [72]:
# data_preprocess[data_preprocess["Loan_Status"] == "X"]
trn_data = model_data.iloc[:480]
tst_data = model_data.iloc[480:]

In [98]:
X_train, X_test, y_train, y_test = train_test_split(trn_data.values, data_preprocess["Loan_Status"].iloc[:480].values,  test_size=0.2, random_state= 42)
xgbRer = XGBClassifier(learning_rate=0.02, n_estimators=100, random_state=42, booster="gbtree", eta=0.4, max_depth=20, scale_pos_weight=1,
 seed=27)
xgbRer.fit(X_train, y_train)
print(xgbRer.score(X_train, y_train))
xgbRer.score(X_test, y_test)

0.9192708333333334


0.8229166666666666

In [102]:
lr = RandomForestClassifier()
lr = GridSearchCV(lr,{"n_estimators":[10, 200], "max_features":[2,3,4,5, 8, 10]},cv=5)
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

0.8125

In [92]:
confusion_matrix(y_test, xgbRer.predict(X_test))
# xgbRer.predict(tst_gdr_data.values)

array([[15, 13],
       [10, 58]], dtype=int64)

In [99]:
xgbRer.predict(X_test)

array(['Y', 'Y', 'Y', 'N', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'N', 'N', 'Y', 'N', 'N', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'N', 'N', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N',
       'Y', 'Y', 'Y', 'Y', 'Y'], dtype=object)

In [107]:
df_test["Loan_Amount_Term"].isnull().sum()

6